### Using OpenCV's dnn.blobFromImage to perform Neural Style Transfer on an input image

In [1]:
from os import listdir
from os.path import isfile, join
import cv2

# Load our t7 neural transfer models
model_file_path = "./models/"
model_file_paths = [f for f in listdir(model_file_path) if isfile(join(model_file_path, f))]

# Load our test image
img = cv2.imread("images/n2.jpg")

# Loop through and applying each model style our input image
for (i,model) in enumerate(model_file_paths):
    # print the model being used
    print(str(i+1) + ". Using Model: " + str(model)[:-3])    
    style = cv2.imread("./art/"+str(model)[:-3]+".jpg")
    # loading our neural style transfer model 
    neuralStyleModel = cv2.dnn.readNetFromTorch(model_file_path+ model)

    # Let's resize to a fixed height of 640 (feel free to change)
    height, width = int(img.shape[0]), int(img.shape[1])
    newWidth = int((640 / height) * width)
    resizedImg = cv2.resize(img, (newWidth, 640), interpolation = cv2.INTER_AREA)

    # Create our blob from the image and then perform a forward pass run of the network
    inpBlob = cv2.dnn.blobFromImage(resizedImg, 1.0, (newWidth, 640),
                               (103.939, 116.779, 123.68), swapRB=False, crop=False)

    neuralStyleModel.setInput(inpBlob)
    output = neuralStyleModel.forward()

    # Reshaping the output tensor, adding back  the mean subtraction 
    # and re-ordering the channels 
    output = output.reshape(3, output.shape[2], output.shape[3])
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.68
    output /= 255
    output = output.transpose(1, 2, 0)
    
    #Display our original image, the style being applied and the final Neural Style Transfer
    cv2.imshow("Original", img)
    cv2.imshow("Style", style)
    cv2.imshow("Neural Style Transfers", output)
    cv2.waitKey(0)

cv2.destroyAllWindows()

1. Using Model: la_muse
2. Using Model: feathers
3. Using Model: composition_vii
4. Using Model: mosaic
5. Using Model: the_scream
6. Using Model: udnie
7. Using Model: starry_night
8. Using Model: candy
9. Using Model: the_wave


### Applying this to our webcam

In [6]:
import cv2
import numpy as np

# Our sketch generating function
def NeuralStyleTransfer(img, model, size = 320, upscale = 1):
    
    model_file_path = "./models/" 
    style = cv2.imread("./art/"+str(model)[:-3]+".jpg")
    # loading our neural style transfer model 
    neuralStyleModel = cv2.dnn.readNetFromTorch(model_file_path+ model+".t7")

    # Let's resize to a fixed height of 640 (feel free to change)
    height, width = int(img.shape[0]), int(img.shape[1])
    newWidth = int((size / height) * width)
    resizedImg = cv2.resize(img, (newWidth, size), interpolation = cv2.INTER_AREA)

    # Create our blob from the image and then perform a forward pass run of the network
    inpBlob = cv2.dnn.blobFromImage(resizedImg, 1.0, (newWidth, size),
                               (103.939, 116.779, 123.68), swapRB=False, crop=False)

    neuralStyleModel.setInput(inpBlob)
    output = neuralStyleModel.forward()

    # Reshaping the output tensor, adding back  the mean subtraction 
    # and re-ordering the channels 
    output = output.reshape(3, output.shape[2], output.shape[3])
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.68
    output /= 255
    output = output.transpose(1, 2, 0)
    output = cv2.resize(output, None, fx=upscale, fy=upscale, interpolation = cv2.INTER_LINEAR)
    return output


# Initialize webcam, cap is the object provided by VideoCapture
cap = cv2.VideoCapture(0)

# la_muse, feathers, composition_vii, mosaic, the_scream, udnie, starry_night, candy, the_wave

while True:
    ret, frame = cap.read()
    cv2.imshow('Neural Style Transfers', NeuralStyleTransfer(frame, "mosaic", 320, 2))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
# Release camera and close windows
cap.release()
cv2.destroyAllWindows()      